### This is the Notebook for Lecture 24 - Logic Design - Combinational Logic

In this lecture, we will learn
<ol>
    <li>Implement Basic Logic Gates</li>
    <li>Develop and test a 1-bit Full Adder</li>
    <li>Use Recursion to develop a 4-bit Full Adder with connected gates</li>
</ol>

In order to properly simulate the output of a circuit, you will need to develop <b>logic blocks</b> that build upon each other. This approach is the hardware equivalent of using <b>functions</b> in software design!

In [1]:
# You will do this library import with every PyRTL assignment
from pyrtl import *
import pyrtl

In [2]:
def two_input_and( a_input, b_input ):
    
    # Security to ensure the lengths passed to the block are 1
    assert len(a_input) == len(b_input)  == 1  
    
    # Create the wire out and put a & b on that 
    y_output = a_input & b_input

    # Use assert to ensure that the signals are one bit
    return y_output

In [3]:
def two_input_and_simulate():

    # Step 1 - Reset the working block
    pyrtl.reset_working_block()

    # Step 2 - Create the input and ouput wires
    a, b = pyrtl.Input(1, 'a'), pyrtl.Input(1, 'b')
    out = pyrtl.Output(1, 'out')

    # Step 3 - Save to an intermediate value using the function
    out_inter = two_input_and( a, b )
    
    # Step 3b - Connect them to the outputs using the <<= operator
    out <<= out_inter
    
    # Step 4 - Start the design simulation
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs = [0,0,1,1]
    b_inputs = [0,1,0,1]
    
    # Step 6 - Loop through and simuluate
    for value in range(0, 4):

        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value]
        })
    
    # Step 7- Render the trace
    sim.tracer.render_trace()

In [4]:
# Call the function
two_input_and_simulate()

<IPython.core.display.Javascript object>

### Now we can combine circuits using a larger block and multiple outputs!

In [5]:
def two_input_or( a_input, b_input ):
    
    # Security to ensure the lengths passed to the block are 1
    assert len(a_input) == len(b_input)  == 1  
    
    # Create the wire out and put a | b on that 
    y_output = a_input | b_input

    # Use assert to ensure that the signals are one bit
    return y_output


def two_input_xor( a_input, b_input ):
    
    # Security to ensure the lengths passed to the block are 1
    assert len(a_input) == len(b_input)  == 1  
    
    # Create the wire out and put a ^ b on that 
    y_output = a_input ^ b_input

    # Use assert to ensure that the signals are one bit
    return y_output

In [6]:
def two_input_or_xor_simulate():

    # Step 1 - Reset the working block
    pyrtl.reset_working_block()

    # Step 2 - Create the input and output wires
    a, b = pyrtl.Input(1, 'a'), pyrtl.Input(1, 'b')
    out_or = pyrtl.Output(1, 'out_or')
    out_xor = pyrtl.Output(1, 'out_xor')

    # Step 3 - Save to an intermediate value using the function(s)
    out_or_inter = two_input_or( a, b )
    out_xor_inter = two_input_xor( a, b )
    
    # Step 3b - Connect them to the outputs using the <<= operator
    out_or <<= out_or_inter
    out_xor <<= out_xor_inter
    
    # Step 4 - Start the design simulation
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs = [0,0,1,1]
    b_inputs = [0,1,0,1]
    
    # Step 6 - Loop through and simuluate
    for value in range(0, 4):
        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value]
        })
        
    # Step 7- Render the trace
    sim.tracer.render_trace()

In [7]:
two_input_or_xor_simulate()

<IPython.core.display.Javascript object>

### Now let's develop a logic circuit with a 3-input AND and 2-input OR

In [8]:
def three_input_and_two_input_or( a_input, b_input, c_input ):
    
    # Security to ensure the lengths passed to the block are 1
    assert len(a_input) == len(b_input) == len(c_input) == 1  
    
    # Calculate the and output
    output_and = a_input & b_input & c_input

    # Calculate the or output
    output_or = two_input_or( a_input, b_input )
    
    # Return both
    return output_and, output_or

In [9]:
def three_input_and_two_input_or_simulate():
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # Step 2 - Create the input and ouput wires
    a, b, c = pyrtl.Input(1, 'a'), pyrtl.Input(1, 'b'), pyrtl.Input(1, 'c')
    output_3_and = pyrtl.Output(1, 'output_3_and')
    output_2_or = pyrtl.Output(1, 'output_2_or')
    
    # Step 3-a - Save to an intermediate value using the three_input_and_or function
    inter_3_and, inter_2_or = three_input_and_two_input_or( a, b, c )
    
    # Step 3-b Assign to a wire using <<=
    output_3_and <<= inter_3_and
    output_2_or <<= inter_2_or
    
    # Step 4 - Simulate the design
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs = [0,0,0,0,1,1,1,1]
    b_inputs = [0,0,1,1,0,0,1,1]
    c_inputs = [0,1,0,1,0,1,0,1]
    
    # Step 6 - Loop through and simuluate
    for value in range(0, 8):

        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value],
            'c' : c_inputs[value] 
        })
    
    # Step 7 - Render the trace
    sim.tracer.render_trace()

In [10]:
three_input_and_two_input_or_simulate()

<IPython.core.display.Javascript object>

## Classic Advanced Circuit: Adder

Now we will build the full adder that we developed together in class!

The logic <b>equations</b> are as follows:
<ol>
    <li><code>Sum = a ^ b ^ Cin</code></li>
    <li><code>Cout = (a & b) | (Cin & (a ^ b))</code></li>
</ol>

The <b>logic diagram</b> and <b>truth table</b> are presented below:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Full-adder_logic_diagram.svg/400px-Full-adder_logic_diagram.svg.png"></center> <br />

| A | B | Cin | Sum | Cout |
|---|---|---|---|---|
|0|0|0|0|0|
|0|0|1|1|0|
|0|1|0|1|0|
|0|1|1|0|1|
|1|0|0|1|0|
|1|0|1|0|1|
|1|1|0|0|1|
|1|1|1|1|1|

In [11]:
def sum( a, b, c_in ):
    
    # Security to ensure the lengths passed to the block are 1
    assert len(a) == len(b) == len(c_in) == 1  
    
    # Return the sum using previously design cells
    # Could also simply do a ^ b ^ cin
    return two_input_xor( two_input_xor(a, b), c_in )

In [12]:
def carry_out( a, b, c_in ):
    
    # Security to ensure the lengths passed to the block are 1
    assert len(a) == len(b) == len(c_in) == 1  
    
    # Return the carry out
    return (a & b) | ( c_in & ( a ^ b ) )

In [13]:
def full_adder( a, b, c_in ):
    
    # Security to ensure the lengths passed to the block are 1
    assert len(a) == len(b) == len(c_in) == 1  
    
    # Calculate the sum
    output_sum = sum( a, b, c_in )

    # Calculate the carry out
    output_c_out = carry_out( a, b, c_in )
    
    # Return both
    return output_sum, output_c_out

In [14]:
def full_adder_simulate():
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # Step 2 - Create the input and ouput wires
    a, b, c_in = pyrtl.Input(1, 'a'), pyrtl.Input(1, 'b'), pyrtl.Input(1, 'c_in')
    output_sum = pyrtl.Output(1, 'output_sum')
    output_c_out = pyrtl.Output(1, 'output_c_out')
    
    # Step 3-a - Save to an intermediate value using the three_input_and_or function
    inter_sum, inter_c_out = full_adder( a, b, c_in )
    
    # Step 3-b Assign to a wire using <<=
    output_sum <<= inter_sum
    output_c_out <<= inter_c_out
    
    # Step 4 - - Simulate the design
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs = [0,0,0,0,1,1,1,1]
    b_inputs = [0,0,1,1,0,0,1,1]
    c_in_inputs = [0,1,0,1,0,1,0,1]
    
    # Step 6 - Loop through and simuluate
    for value in range(0, len(a_inputs)):

        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value],
            'c_in' : c_in_inputs[value] 
        })
    
    # Render the trace
    sim.tracer.render_trace()

In [15]:
# Call the function
full_adder_simulate()

<IPython.core.display.Javascript object>

### Combining Concepts: Using Recursion to Create a Full Adder

When we create a circuit in PyRTL, we cannot use loop to create the signals, only the input signals. But if there is repetition, we can use <b>recursion</b>!

In [16]:
def ripple_carry_adder( a, b, c_in ):
    
    # Check inputs a must be the same size as b, and c must be 1 bit
    assert len(a) == len(b)
    assert len(c_in) == 1
    
    if len(a) == 1:
        sum_result, c_out = full_adder( a, b, c_in )
        
    else:

        # Get the next smallest bit
        ls_bit, ripplecarry = full_adder(a[0], b[0], c_in)
        
        # Recursively call the remaining bits
        ms_bits, c_out = ripple_carry_adder(a[1:], b[1:], ripplecarry)
        
        # Concatenate the results on the recursive call back up
        sum_result = pyrtl.concat(ms_bits, ls_bit)

                                          
    return sum_result, c_out

In [17]:
def ripple_carry_sim( ):
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # Step 2 - Create the input and ouput wires
    a, b, c_in = pyrtl.Input(4, 'a'), pyrtl.Input(4, 'b'), pyrtl.Input(1, 'c_in')
    
    # WireVector Example!
    output_sum = pyrtl.Output(4, 'output_sum')
    
    # c_out is one bit, so we can still use .Output without issue
    output_c_out = pyrtl.Output(1, 'output_c_out')
    
    # Step 3-a - Save to an intermediate value using the three_input_and_or function
    inter_sum, inter_c_out = ripple_carry_adder( a, b, c_in )
    
    # Step 3-b Assign to a bus wire using <<=
    output_sum <<= inter_sum
    output_c_out <<= inter_c_out
    
    # Step 4 - Simulate the design
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    a_inputs = [ 0, 3, 10, 2, 15, 3, 14 ]
    b_inputs = [ 4, 6, 6, 11, 15, 12, 1 ]
    c_in_inputs = [0, 0, 0, 0, 0, 0, 0]

    # Step 6 - Loop through and simuluate
    for value in range( 0, len(a_inputs) ):

        sim.step({
            'a' : a_inputs[value],
            'b' : b_inputs[value],
            'c_in' : c_in_inputs[value] 
        })
    
    # Render the trace
    sim.tracer.render_trace()

In [18]:
ripple_carry_sim( )

<IPython.core.display.Javascript object>